# Uninformed Tree Search Without Duplicates Detection

In this lab, you will implement basic tree search methods without duplicate detection:

- ***BFS (Breadth First Search)***
- ***DFS (Depth First Search)***
- ***DFS-L (Depth First Search with Limited Depth)***
- ***DFID (Depth First Iterative Deepening)***.

You will work with two widespread domains:
- ***15-puzzle***
- ***Panckakes***.

For the ***15-puzzle***, the code that defines `state` and `get_successors` is already available. For Pancakes, you will need to code it yourself. All search methods must be implemented by you using the provided code stubs.

Run every cell of the notebook and complete the described tasks. Good luck!

In [2]:
import copy
from pathlib import Path
from collections import deque
from typing import List, Callable

import numpy as np

## Gem Puzzle (15-puzzle or n-puzzle)

The ***15-puzzle*** (also called ***Gem Puzzle***, Boss Puzzle, Game of Fifteen, Mystic Square and many others) is a sliding puzzle that consists of a frame of numbered square tiles in random order with one tile missing (the so-called blank tile).

The size of the puzzle may vary. E.g. if the size of the field is $3\times3$, the puzzle is called the 8-puzzle, and if the size of the field is $4 \times 4$, the puzzle is called the 15-puzzle.

The task is to place the tiles in order (see the figure below) by sliding them. Indeed, you can slide a tile only to an adjacent blank tile, which is why one can think of it as actually sliding the blank tile.

Note, that half of the initial configurations for the n-puzzle are impossible to resolve, no matter how many moves are made. See [[Wikipedia](https://en.wikipedia.org/wiki/15_puzzle)] for more info.

![puzzle](img/gem_puzzle.png)

### Representation of a search state for the Gem Puzzle

Indeed, there may exist many ways to represent a search state for the Gem Puzzle. In this lab, we will use a list of integers as an external encoding of the Gem Puzzle state. This list is assumed to contain numbers from 1 to (*size* * *size*), where *size* is the size of the puzzle. The tile with the number *size* * *size* is a blank tile.

For example, the encoding of the start state of the 8-puzzle depicted above will be \[7,2,4,5,9,6,8,3,1\].

Internally, we will use a 2-dimensional numpy array to store puzzle states.


In [65]:
class GemPuzzleState:
    """
    Implementing a search state (or simply, a state) in code is a crucial first step
    necessary for tackling any search problem. The `GemPuzzleState` class is structured
    with the following fields:

    Attributes
    ----------
    size : int
        Width of the game field.

    tile_matrix : ndarray[int, ndim=2]
        Tile positions represented as a 2D array of integers. This array is expected
        to contain values from `1` to `size * size`. Each integer value corresponds
        to a tile, and the position in the array (row and column) corresponds to the
        position of the tile on the game field. The tile with the value `size * size`
        is assumed to represent the blank position.

    parent : GemPuzzleState
        A pointer to the parent state. The parent is a predecessor of the state in
        the search tree. It is used to reconstruct a path to that state from the start
        state (the root of the search tree).

    blank_pos : ndarray[int, ndim=1]
        The position (row and column) of the empty tile in tile_matrix. Explicitly
        storing the position of a blank helps to generate successors faster.
    """

    def __init__(self, tile_list: List[int]):
        """
        Constructor. Sets tile positions and performs some basic checks.

        Parameters
        ----------
        tile_list : List[int]
            Tile positions represented as a list of integers. This list is expected to contain
            values from `1` to `size * size`. Each integer value corresponds to a tile, and
            the position in the list (index) corresponds to the position of the tile on the game field.
            The tile with the value `size * size` is assumed to represent the blank position.
        """
        self.size = int(len(tile_list) ** 0.5)
        blank_value = self.size**2
        if blank_value != len(tile_list):
            raise ValueError(
                "The tile list must contain the number of elements equal to the square of an integer!"
            )

        self.tile_matrix = np.array(tile_list, dtype=np.int16).reshape(
            (self.size, self.size)
        )

        # Memorizing the position of a blank tile
        # Technically, there is no need to do so,
        # but it makes getting the successors a bit faster
        blanks = np.argwhere(self.tile_matrix == blank_value)
        if len(blanks) != 1:
            raise ValueError(
                "The state should contain a single maximum value representing the blank tile position."
            )
        self.blank_pos = blanks[0]

        # The parent state (predecessor in the search tree) will be set up by the search algorithm.
        self.parent = None
        self.state = self.tile_matrix.flatten()
        self.needed_state =  np.arange(1, self.size**2 + 1)

    def __eq__(self, other):
        """
        Compare one state with another state.
        """
        return np.array_equal(self.tile_matrix, other.tile_matrix)

    def __str__(self) -> str:
        """
        Return a string representation of the game field for printing.
        """
        blank_value = self.size**2
        result = (
            str(self.tile_matrix)
            .replace(" [", "")
            .replace("[", "")
            .replace("]", "")
            .replace(str(blank_value), "_")
            + "\n"
        )
        return result

### Get Succesors

In [4]:
def get_successors(state: GemPuzzleState) -> List[GemPuzzleState]:
    """
    Implementing the `get_successors` function is another crucial step in tackling any search problem.
    This function is designed to take a specific search state as input and return all possible successor states,
    which result from applying all applicable actions to the input state. In the case of GemPuzzle, the successors
    correspond to the board states resulting from moving the blank tile up, down, left, or right. If the blank tile
    goes out of the field after a move, such a successor should be discarded.

    Parameters
    ----------
    state : GemPuzzleState
        The input search state.

    Returns
    -------
    List[GemPuzzleState]
        A list containing all possible successor states for the input state.
    """
    successors = []
    row, col = state.blank_pos
    delta = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    for dx, dy in delta:
        # Compute new column and row for the blank tile
        new_row, new_col = row + dx, col + dy

        # If the new position of the blank tile is valid (i.e., it is still within the field), then
        # a corresponding successor should be added to the successors' list
        if 0 <= new_row < state.size and 0 <= new_col < state.size:
            new_state = copy.deepcopy(state)
            new_state.tile_matrix[row, col] = new_state.tile_matrix[new_row, new_col]
            new_state.tile_matrix[new_row, new_col] = new_state.size**2
            new_state.blank_pos = (new_row, new_col)
            new_state.parent = state
            successors.append(new_state)

    return successors

### Goal check

In [5]:
def state_is_goal(state: GemPuzzleState):
    """
    A handy function that returns `True` if the input `state` corresponds
    to the goal state (i.e., all tiles are in their correct positions), and `False` otherwise.
    """
    goal_list = np.arange(1, state.size**2 + 1).reshape(state.size, state.size)
    return np.array_equal(state.tile_matrix, goal_list)

### Path checking

In [6]:
def check_path(last_state):
    """
    Auxiliary function that takes the `last_state` and checks whether this state is a goal.
    If yes, it unwinds the path using the backpointers and checks whether each successor is
    reachable from its predecessor.
    """
    curr = last_state
    if not state_is_goal(curr):
        print("Goal was not reached!")
        return False

    while curr.parent is not None:
        prev = curr.parent
        if curr not in get_successors(prev):
            print("Unacceptable step!")
            return False
        curr = prev
    return True

### Path unwinding
Typically the paths are not stored within a search explicitly, but rather implicitly via the parent pointers (pointing to the predecessor in the search tree). Thus, when we reach the goal state and want to reconstruct the whole path we need to trace the parent pointers back to the root of the tree.

In [7]:
def get_path(last_state: GemPuzzleState):
    """
    Takes an input state `last_state` and returns a path to this state from the root of the tree.
    """
    path = []
    curr = last_state
    while curr is not None:
        path.append(curr)
        curr = curr.parent
    return path

## Automated Tests to Check the Implementations of the Search Algorithms
When you finish implementing search algorithms you need to test them, right? The following functions will help you in that. They take your search algorithm as an input and run it on a single simple test (`simple_test`) and on a series of more involved tests (`massive_test`).

These automated tests assume that the seach function, passed as the input, has the following structure:

`search_function(start_state, *optional arguments*) -> (path_found, last_state)`, where

- `start_state` — initial state
- `*optional arguments*` — additional parameters of the search function (if needed), passed via `*args`
- `path_found` — result of the search, `True` if path was found, `False` otherwise
- `last_state` — last state of path. `None` if path was not found

In [66]:
def _run_test(
    search_function: Callable, state, needed_state, *args, task_idx: int = 0, print_path: bool = True
):
    """
    Execute a test using the specified search function on the given state.

    Parameters
    ----------
    search_function : Callable
        The search function to be tested.
    state
        The initial state for the test.
    *args
        Additional arguments for the search function.
    task_idx : int, optional
        The index of the task being tested (default is 0).
    print_path : bool, optional
        If True, print the path found by the search function (default is True).

    Returns
    -------
    None
    """
    try:
        path_found, curr = search_function(state, needed_state, *args)
        if not path_found:
            print(f"Task #{task_idx}: path not found :(")
            return
        if check_path(curr):
            path = get_path(curr)
            print(f"Task #{task_idx}: path is OK! Path length: {len(path)}")
            if not print_path:
                return
            while len(path) != 0:
                print(path.pop())
        else:
            print(f"Task #{task_idx}: path is not OK :(")

    except Exception as e:
        print(f"Task #{task_idx}: execution error {e}")

In [67]:
def simple_test(search_function: Callable, *args, **kwargs):
    """
    The `simple_test` function runs the `search_function` on a simple 2 x 2 sliding
    puzzle instance (encoded as [3, 1, 2, 4]).

    Parameters
    ----------
    search_function : Callable
        Implementation of the search method.
    *args
        Additional arguments for the search function.
    **kwargs
        Additional keyword arguments for the search function.

    Possible Outputs:
    - 'path is OK!':
        A path was found, and it is correct.
    - 'path is not OK :(':
        A path was found, but it is not correct.
    - 'path not found :(':
        A path was not found.
    - 'execution error':
        An error occurred while executing the `search_function` or path validation function.

    Additional Parameters (optional):
    - task_idx : int, optional
        The index of the task being tested (default is 0).
    - print_path : bool, optional
        If True, print the path found by the search function (default is True).
    """
    start_state = GemPuzzleState([3, 1, 2, 4])
    needed_state = GemPuzzleState([1, 2, 3, 4])
    _run_test(search_function, start_state, needed_state, *args, **kwargs)

In [68]:
def massive_test(search_function: Callable, *args, **kwargs):
    """
    The `massive_test` function runs the `search_function` on a set of different tasks stored in `data/tasks_gem.txt`.
    Initially, this file contains 4 different 2 x 2 sliding puzzle instances and 4 different 3 x 3
    sliding puzzle instances (more can be added if desired). Each instance starts on a new line and
    is represented as a sequence of integers separated by spaces.

    The output is similar to `simple_test`, but explicit paths for the solved instances are not displayed:

    Possible Outputs:
    - 'path is OK!':
        A path was found, and it is correct.
    - 'path is not OK :(':
        A path was found, but it is not correct.
    - 'path not found :(':
        A path was not found.
    - 'execution error':
        An error occurred while executing the `search_function` or path validation function.

    Parameters
    ----------
    search_function : Callable
        Implementation of the search method.
    *args
        Additional arguments for the search function.
    **kwargs
        Additional keyword arguments for the search function.
        - print_path : bool, optional
            If True, print the path found by the search function (default is False).
    """
    with open(Path("data/tasks_gem.txt")) as tasks_file:
        print_path = kwargs.pop("print_path", False)
        for count, line in enumerate(tasks_file):
            state = list(map(int, line.split()))
            task = GemPuzzleState(state)
            needed = GemPuzzleState(np.arange(1, task.size**2 + 1))
            _run_test(
                search_function, task, needed, task_idx=count + 1, print_path=print_path, *args
            )

## Search Algorithms Implementation


Recall again here that *dublicate detection must not be coded* in this assignment.

### Breadth-First Search (BFS)

In [124]:
def bfs(start, needed_state):
    """
    Implementation of Breadth-First Search algorithm.
    Returns (path_found, last_state).
    """
    # in case target and start are same
    if start == needed_state:
      return True, needed_state

    stack = [start]

    while stack:
      cur_state = stack.pop(0)

      if needed_state == cur_state:
        return True, cur_state

      # add new states from current to the end of list
      stack += get_successors(cur_state)

    return False, None

In [70]:
# Test your BFS on simple task
simple_test(bfs)

Task #0: path is OK! Path length: 5
3 1
2 _

3 1
_ 2

_ 1
3 2

1 _
3 2

1 2
3 _



In [71]:
# If simple test is path_found, you should check your implementation in massive test.
# The rest of the search algorithms are checked in the same way.
massive_test(bfs)

Task #1: path is OK! Path length: 5
Task #2: path is OK! Path length: 5
Task #3: path is OK! Path length: 3
Task #4: path is OK! Path length: 4
Task #5: path is OK! Path length: 4
Task #6: path is OK! Path length: 6
Task #7: path is OK! Path length: 5
Task #8: path is OK! Path length: 9


### Depth-First Search (DFS)

In [74]:
def dfs(start, needed_state):
    """
    Implementation of Depth-First Search algorithm.
    Returns (path_found, last_state).
    Warning: No duplicate detection. May recurse deeply.
    """

    stack = [start]

    while stack:
      # take the 1st element
      cur_state = stack.pop(0)

      if needed_state == cur_state:
          return True, cur_state

      # get successors and add them to the beginning of the stack
      successors = get_successors(cur_state)
      stack = successors + stack

    return False, None

Using DFS, you will most likely encounter the fact that this algorithm overcomes the threshold of recursive calls, after which the execution will interrupted.

Other unpleasant outcomes (e.g. dead kernel) are also possible.

In [75]:
simple_test(dfs)

Task #0: execution error maximum recursion depth exceeded


In [ ]:
# There is no need to start MassiveTest
# massive_test(dfs)

But you can create such a simple task, which can be solved by DFS. So your task is to create an instance of the 8-puzzle that is solvable by DFS and the solution contains *at least 3 moves*.

In [80]:
def dfs_simple_test(search_function, *args):
    # TODO: Create your simple test
    your_tile_list = [1,4,3,2] # Created test when we win in 1 step
    start_state = GemPuzzleState(your_tile_list)
    needed_state = GemPuzzleState([1, 2, 3, 4])
    _run_test(search_function, start_state, needed_state, print_path=True, *args)

In [81]:
dfs_simple_test(dfs)

Task #0: path is OK! Path length: 2
1 _
3 2

1 2
3 _



### Depth-First Search (DFS) with Random Choice

You may try to increase the chance of solving the input problem by randomizing the DFS. One way to do so is to recursively go deeper in the search tree not using the first successor returned by the (deterministic) get_succesors function, but rather by picking a random succesor.

Indeed, this technique does not provide any guarantess and in practice it is likely to fail on numerous instances.

In [125]:
import random

random.seed(42)

def dfs_random(start, needed_state):
    """
    Implementation of Depth-First Search with Random Choice algorithm.
    Returns (path_found, last_state).
    """
    stack = [start]

    while stack:
      # take the 1st element
      cur_state = stack.pop(0)

      if needed_state == cur_state:
          return True, cur_state

      # get successors
      successors = get_successors(cur_state)
      # random shuffle successors
      random.shuffle(successors)
      # add them to the beginning of the stack
      stack = successors + stack

    return False, None

In [83]:
for i in range(10):
    simple_test(dfs_random, task_idx=i)

Task #0: path is OK! Path length: 43
3 1
2 _

3 _
2 1

_ 3
2 1

3 _
2 1

_ 3
2 1

2 3
_ 1

_ 3
2 1

2 3
_ 1

_ 3
2 1

3 _
2 1

_ 3
2 1

2 3
_ 1

_ 3
2 1

2 3
_ 1

_ 3
2 1

2 3
_ 1

_ 3
2 1

3 _
2 1

_ 3
2 1

3 _
2 1

3 1
2 _

3 _
2 1

_ 3
2 1

3 _
2 1

3 1
2 _

3 1
_ 2

_ 1
3 2

3 1
_ 2

3 1
2 _

3 _
2 1

_ 3
2 1

3 _
2 1

_ 3
2 1

3 _
2 1

3 1
2 _

3 1
_ 2

_ 1
3 2

1 _
3 2

_ 1
3 2

1 _
3 2

_ 1
3 2

1 _
3 2

1 2
3 _

Task #1: path is OK! Path length: 57
3 1
2 _

3 _
2 1

_ 3
2 1

2 3
_ 1

_ 3
2 1

3 _
2 1

_ 3
2 1

2 3
_ 1

_ 3
2 1

3 _
2 1

3 1
2 _

3 1
_ 2

3 1
2 _

3 _
2 1

3 1
2 _

3 1
_ 2

_ 1
3 2

1 _
3 2

_ 1
3 2

3 1
_ 2

_ 1
3 2

3 1
_ 2

3 1
2 _

3 1
_ 2

3 1
2 _

3 _
2 1

3 1
2 _

3 _
2 1

_ 3
2 1

2 3
_ 1

2 3
1 _

2 3
_ 1

2 3
1 _

2 _
1 3

_ 2
1 3

2 _
1 3

_ 2
1 3

2 _
1 3

2 3
1 _

2 3
_ 1

_ 3
2 1

3 _
2 1

_ 3
2 1

2 3
_ 1

2 3
1 _

2 3
_ 1

2 3
1 _

2 3
_ 1

_ 3
2 1

2 3
_ 1

2 3
1 _

2 _
1 3

_ 2
1 3

1 2
_ 3

_ 2
1 3

1 2
_ 3

1 2
3 _

Task #2: path is OK! Path 

### Depth First Search with Limited Depth
One of the way to solve problem of overcoming the threshold of recursive calls is explicitly limit the to depth of the search tree by passing an appropriate parameter `limit` to the search algorithm. The second parameter `depth` is a technical one needed for the implementation. It represents the current depth of the search. Initially (when invoked on the start state of the problem) it is, indeed, equal to 0.

In [84]:
def dfs_limited(start, needed_state, limit, depth):
    """
    Implementation of Depth First Search with Limited Depth algorithm.
    Returns (path_found, last_state).
    """

    stack = [start]
    depths = [0]

    while stack:
      # take the 1st element
      cur_state = stack.pop(0)
      cur_depth = depths.pop(0)

      if needed_state == cur_state:
          return True, cur_state

      # end branch if it`s limit
      if cur_depth == limit:
        continue

      # get successors
      successors = get_successors(cur_state)
      # add them to the beginning of the stack
      stack = successors + stack
      depths = [cur_depth + 1] * len(successors) + depths

    return False, None

Let's check this approach with several different limits

In [85]:
simple_test(dfs_limited, 3, 0)

Task #0: path not found :(


In [86]:
simple_test(dfs_limited, 5, 0)

Task #0: path is OK! Path length: 5
3 1
2 _

3 1
_ 2

_ 1
3 2

1 _
3 2

1 2
3 _



In [87]:
simple_test(dfs_limited, 10, 0)

Task #0: path is OK! Path length: 11
3 1
2 _

3 1
_ 2

3 1
2 _

3 1
_ 2

3 1
2 _

3 1
_ 2

3 1
2 _

3 1
_ 2

_ 1
3 2

1 _
3 2

1 2
3 _



In [88]:
massive_test(dfs_limited, 3, 0, print_path=False)

Task #1: path not found :(
Task #2: path not found :(
Task #3: path is OK! Path length: 3
Task #4: path is OK! Path length: 4
Task #5: path is OK! Path length: 4
Task #6: path not found :(
Task #7: path not found :(
Task #8: path not found :(


In [89]:
massive_test(dfs_limited, 5, 0)

Task #1: path is OK! Path length: 5
Task #2: path is OK! Path length: 5
Task #3: path is OK! Path length: 3
Task #4: path is OK! Path length: 6
Task #5: path is OK! Path length: 6
Task #6: path is OK! Path length: 6
Task #7: path is OK! Path length: 5
Task #8: path not found :(


In [90]:
massive_test(dfs_limited, 10, 0)

Task #1: path is OK! Path length: 11
Task #2: path is OK! Path length: 11
Task #3: path is OK! Path length: 3
Task #4: path is OK! Path length: 10
Task #5: path is OK! Path length: 10
Task #6: path is OK! Path length: 10
Task #7: path is OK! Path length: 11
Task #8: path is OK! Path length: 11


### Depth First Iterative Deepening Search (DFID)
Finally let's sequentially invoke DFS with increasing depth limits. This is called the Depth First Iterative Deepening algorithm. It will inded find a solution if one exists.

In [126]:
def dfid(start, needed_state):
    """
    Implementation of Iterative-Deepening Depth-First Search.
    Returns (path_found, last_state).
    """
    cur_limit = 1
    # path can`t be bigger than total states count: (size^2)!
    total_limit = 1
    for i in range(1, start.size ** 2 + 1):
      total_limit *= i

    while cur_limit < total_limit:
      exist, path = dfs_limited(start, needed_state, cur_limit, 0)
      if exist:
        return True, path
      else:
        cur_limit += 1

    return False, None

In [93]:
simple_test(dfid)

Task #0: path is OK! Path length: 5
3 1
2 _

3 1
_ 2

_ 1
3 2

1 _
3 2

1 2
3 _



In [94]:
massive_test(dfid)

Task #1: path is OK! Path length: 5
Task #2: path is OK! Path length: 5
Task #3: path is OK! Path length: 3
Task #4: path is OK! Path length: 4
Task #5: path is OK! Path length: 4
Task #6: path is OK! Path length: 6
Task #7: path is OK! Path length: 5
Task #8: path is OK! Path length: 9


## Pancake Sorting

![Example](img/cat.jpg)

Pancake sorting is the colloquial term for the mathematical problem of sorting a disordered stack of pancakes in order of size when a spatula can be inserted at any point in the stack and used to flip all pancakes above it (See picture below) [[Wikipedia](https://en.wikipedia.org/wiki/Pancake_sorting)].

![Example](img/pancake.png)

### Representation of a state
In this task you should create your own implementation of pancake sorting problem state (and all related funtions) with your own test data. Note, that the interface of the state-class must be the same as for the `GemPuzzleState` thus all the machinery introduced before (e.g. automated tests) will work out-of-the-box.

In [127]:
class PancakeDish:
    def __init__(self, pancakes: List[int]):
        self.pancakes = pancakes
        self.parent = None
        self.size = len(pancakes)

    def __eq__(self, other):
        return self.pancakes == other.pancakes

    def __str__(self):
        return str(self.pancakes)[1:-1]

    def __len__(self):
      return len(self.pancakes)

    def __size__(self):
      return len(self.pancakes)

In [128]:
def get_successors(state: PancakeDish):
    successors = []
    for i in range(2, len(state.pancakes) + 1):
        new_state = copy.deepcopy(state)
        new_state.pancakes[0:i] = new_state.pancakes[i - 1 :: -1]
        new_state.parent = state
        successors.append(new_state)
    return successors

In [129]:
def state_is_goal(state: PancakeDish):
    for i in range(1, len(state.pancakes)):
        if state.pancakes[i - 1] > state.pancakes[i]:
            return False
    return True

In [130]:
def simple_test(search_function: Callable, *args, **kwargs):
    # TODO: Create your simple test
    start_state = PancakeDish([4, 3, 2, 1])
    needed_state = PancakeDish([1, 2, 3, 4])
    _run_test(search_function, start_state, needed_state, *args, **kwargs)

In [131]:
def massive_test(search_function: Callable, *args, **kwargs):
    # Uses a custom file with at least 8 different pancake tasks
    with open(Path("data/tasks_capcake.txt")) as tasks_file:
        print_path = kwargs.pop("print_path", False)
        for count, line in enumerate(tasks_file):
            state = list(map(int, line.split()))
            end_state = list(map(int, line.split()))
            task = PancakeDish(state)
            end_state.sort()
            needed = PancakeDish(end_state)
            _run_test(
                search_function, task, needed, task_idx=count + 1, print_path=print_path, *args
            )

## Lets check!

In [132]:
simple_test(bfs)

Task #0: path is OK! Path length: 2
4, 3, 2, 1
1, 2, 3, 4


In [133]:
massive_test(bfs)

Task #1: path is OK! Path length: 3
Task #2: path is OK! Path length: 3
Task #3: path is OK! Path length: 4


In [134]:
simple_test(dfs)

Task #0: execution error maximum recursion depth exceeded


In [135]:
for i in range(10):
    simple_test(dfs_random, task_idx=i, print_path=False)

Task #0: path is OK! Path length: 23
Task #1: path is OK! Path length: 82
Task #2: path is OK! Path length: 14
Task #3: path is OK! Path length: 6
Task #4: path is OK! Path length: 2
Task #5: path is OK! Path length: 2
Task #6: path is OK! Path length: 2
Task #7: path is OK! Path length: 35
Task #8: path is OK! Path length: 2
Task #9: path is OK! Path length: 11


In [136]:
simple_test(dfs_limited, 2, 0)
simple_test(dfs_limited, 5, 0)
simple_test(dfs_limited, 10, 0)

Task #0: path is OK! Path length: 2
4, 3, 2, 1
1, 2, 3, 4
Task #0: path is OK! Path length: 6
4, 3, 2, 1
3, 4, 2, 1
4, 3, 2, 1
3, 4, 2, 1
4, 3, 2, 1
1, 2, 3, 4
Task #0: path is OK! Path length: 10
4, 3, 2, 1
3, 4, 2, 1
4, 3, 2, 1
3, 4, 2, 1
4, 3, 2, 1
3, 4, 2, 1
4, 3, 2, 1
3, 4, 2, 1
4, 3, 2, 1
1, 2, 3, 4


In [137]:
massive_test(dfs_limited, 2, 0)

Task #1: path is OK! Path length: 3
Task #2: path is OK! Path length: 3
Task #3: path not found :(


In [138]:
massive_test(dfs_limited, 5, 0)

Task #1: path is OK! Path length: 5
Task #2: path is OK! Path length: 5
Task #3: path is OK! Path length: 6


In [139]:
massive_test(dfs_limited, 10, 0)

Task #1: path is OK! Path length: 11
Task #2: path is OK! Path length: 11
Task #3: path is OK! Path length: 10


In [140]:
simple_test(dfid)

Task #0: path is OK! Path length: 2
4, 3, 2, 1
1, 2, 3, 4


In [141]:
massive_test(dfid)

Task #1: path is OK! Path length: 3
Task #2: path is OK! Path length: 3
Task #3: path is OK! Path length: 4
